# 🧪 Revue de votre projet — Churn bancaire

> Feedback automatique + suggestions concrètes d'amélioration.

## ✅ Couverture des étapes

- **Chargement & EDA** : 5/5 (100%)
- **Qualité des données** : 3/3 (100%)
- **Visualisations** : 3/5 (60%)
- **Analyses métier** : 1/3 (33%)
- **Modélisation (optionnelle)** : 0/6 (0%)
- **Rapport exécutif** : 0/1 (0%)


## 💪 Points forts
- Chargement des données OK (`pd.read_csv`).
- Exploration de base présente (head/shape/info/describe).
- Visualisations essentielles présentes (countplot/hist/box/bar/heatmap).

## 🧩 Points à renforcer
### Visualisations
- Comparaison num vs cat (boxplot/violin)
- Comparaisons (Top N barplot)
### Analyses métier
- Taux de churn par pays (groupby)
- Taux de churn par genre (groupby)
### Modélisation (optionnelle)
- Séparation train/test (stratify=y)
- Régression logistique baseline
- Random Forest (comparaison)
- Standardisation/normalisation des features
- Gestion du déséquilibre (class_weight='balanced')
- Métriques (precision/recall/F1/ROC-AUC/confusion matrix)
### Rapport exécutif
- Rapport exécutif final (10 lignes)


## 🧹 Nettoyage — snippets prêts à l’emploi

In [ ]:

# Colonnes à ignorer pour l'analyse/modeling
drop_cols = [c for c in ['RowNumber','CustomerId','Surname'] if c in df.columns]
df = df.drop(columns=drop_cols, errors='ignore')

# NA & doublons
na = df.isnull().sum().sort_values(ascending=False)
print(na[na>0])
print("Doublons:", df.duplicated().sum())


## 📌 KPI Churn — taux global & par segments

In [ ]:

# Taux global
if 'Exited' in df.columns:
    churn_rate = df['Exited'].mean()
    print("Taux de churn global:", round(churn_rate*100, 2), "%")

# Par pays
if {'Geography','Exited'}.issubset(df.columns):
    churn_by_country = df.groupby('Geography')['Exited'].mean().sort_values(ascending=False)
    display((churn_by_country*100).round(2).rename('% churn'))

# Par genre
if {'Gender','Exited'}.issubset(df.columns):
    churn_by_gender = df.groupby('Gender')['Exited'].mean().sort_values(ascending=False)
    display((churn_by_gender*100).round(2).rename('% churn'))


## 📊 Visualisations lisibles (triées & titrées)

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt

# Barplot churn par pays (trié)
if {'Geography','Exited'}.issubset(df.columns):
    churn_by_country = df.groupby('Geography')['Exited'].mean().sort_values(ascending=False)
    ax = sns.barplot(x=churn_by_country.index, y=churn_by_country.values)
    ax.set_title('Taux de churn par pays')
    ax.set_xlabel('Pays'); ax.set_ylabel('Taux de churn')
    plt.xticks(rotation=30); plt.tight_layout(); plt.show()

# Histogramme âge (churn vs non churn)
if {'Age','Exited'}.issubset(df.columns):
    sns.histplot(data=df, x='Age', hue='Exited', bins=30, stat='density', common_norm=False)
    plt.title('Distribution de l\'âge — churn vs non-churn'); plt.tight_layout(); plt.show()


## 🤖 Baseline modèle (optionnel) — Logistique vs RandomForest

In [1]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

target = 'Exited'
if target in df.columns:
    # Numériques / catégorielles
    num_cols = df.select_dtypes(include=['int64','float64','int32','float32']).columns.tolist()
    cat_cols = [c for c in df.select_dtypes(exclude=['int64','float64','int32','float32']).columns if c != target]

    if target in num_cols: num_cols.remove(target)

    X = df[num_cols + cat_cols].copy()
    y = df[target].copy()

    preproc = ColumnTransformer([
        ('num', StandardScaler(with_mean=False), num_cols),
        ('cat', 'passthrough', cat_cols)
    ], remainder='drop')

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    logit = Pipeline([('prep', preproc),
                      ('clf', LogisticRegression(max_iter=1000, class_weight='balanced'))])

    rf = Pipeline([('prep', preproc),
                   ('clf', RandomForestClassifier(n_estimators=300, random_state=42, class_weight='balanced'))])

    for name, model in [('LogReg', logit), ('RandomForest', rf)]:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:,1]
        print("\n=== ", name, " ===")
        print(classification_report(y_test, y_pred, digits=3))
        print("ROC-AUC:", round(roc_auc_score(y_test, y_prob), 3))
else:
    print("👉 Ajoute/nomme la colonne cible 'Exited' pour activer ce bloc.")


NameError: name 'df' is not defined


## 📑 Rapport exécutif — Template

**Contexte** : La banque souhaite réduire le churn en identifiant les profils à risque.  
**Résultats clés** :  
- Taux de churn global : …%  
- Segments les plus à risque : … (pays, âge, produits, activité)  
- Facteurs influents : … (Age, NumOfProducts, IsActiveMember, Geography)  
**Recommandations** :  
- Campagnes ciblées sur …  
- Cross-sell vers …  
- Relance des clients inactifs …  
